<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Machine_Replacement_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 76.0 MB/s eta 0:00:00


In [2]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('equipment maintainance')

# indices for companies and routes
node = [*range(0,21)]
start = [*range(0,1)]
transit = [*range(1,20)]
end = [*range(20,21)]

#print(node)
#print(start)
#print(transit)
#print(end)

# various cost/price

p = 3530
s0 = 1530
s1 = 110
m0 = 100
m1 = 65

# build c_ij matrix

c = [[0 for i in node] for j in node]

# Valid set of tuples
A = []
for i in node:
    for j in node:
        if j-i >= 4 and j-i <= 11:

            mc = 0
            for k in range(0,j-i):
              mc += m0+m1*k

            c[i][j] = p + mc - (s0-s1*(j-i-1))

            tp = i,j
            A.append(tp)

print(np.matrix(c))

# Valid set of tuples
#A = []
#for i in node:
#    for j in node:
#        if c[i][j] > 0:
#            tp = i,j
#            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of inbound routes for node j
AI = []
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AI[0]))

# valid set of outbound routes for node j
AO = []
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AO.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AO[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.BINARY, name='Keep')

# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)

# Reach the fifth year (21 quarter)
endConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) >= 1 for j in end),
                                      name='endConstrs')

# Cannot exceed plant capacity
startConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AO[i]) - gp.quicksum(x[(k,i)] for k,i in AI[i]) <= 1 for i in start),
                                      name='startConstrs')

# Blanacing for transit nodes
transitConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) == 0 for j in transit),
                                      name='transitConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
[[   0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775    0    0
     0    0    0    0    0    0    0]
 [   0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775    0
     0    0    0    0    0    0    0]
 [   0    0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775
     0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915
  7775    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0 3120 3590 4125 4725 5390 6120
  6915 7775    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0 3120 3590 4125 4725 5390
  6120 6915 7775    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0 3120 3590 4125 4725
  5390 6120 6915 7775    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0 3120 3590 4125
  4725 5390 6120 6915 7775    0    0]
 [   0    0    0    0    0    0    0    0    0    

In [4]:
#####################################################
#         Shipment results
#####################################################

print(f"\n\n___Optimal shipment from plants to customers________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        if i<=0:
          s_node_type = "start"
        elif i<=19:
          s_node_type = "transit"
        else:
          s_node_type = "end"
        if j<=0:
          d_node_type = "start"
        elif j<=19:
          d_node_type = "transit"
        else:
          d_node_type = "end"

        print("Keep machines from %s quarter %2d to %s quarter %2d" % (s_node_type, i+1, d_node_type, j+1))
        t_cost += x[(i,j)].x*c[i][j]

print("The total cost of equipment maintainance over the five years is $%5d" % (t_cost))



___Optimal shipment from plants to customers________
Keep machines from start quarter  1 to transit quarter  7
Keep machines from transit quarter  7 to transit quarter 14
Keep machines from transit quarter 14 to end quarter 21
The total cost of equipment maintainance over the five years is $13575


##  Conclusion

In this example, we addressed the tomato shipping problem. We determined the optimal shipment  of tomatos from plants to customers:
* Satisfy demand for each customer,
* Minimize the total shipping cost,  
* Ensure plant capacities are not exceeded, and
* Utilize transhipment to reduce shipping cost.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid routes only. This benefit becomes more significant as problem size grows.

This tomato shipment model can be used in many different settings to help companies make informed decisions about satisfying customer demands from a set of plants where there are transit stations allowing for transhipments.
